In [ ]:
import pandas as pd
import numpy as np

valors_socials_cat = pd.read_csv('data/values/collective_values.csv', encoding='windows-1252', delimiter=';')

# variables socio-demogràfiques i laborals
# eliminat: 'DISTRICTE'
columns_to_keep = ['SEXE', 'EDAT', 'CIUTADANIA', 'ESTUDIS_1_6', 'ESTAT_CIVIL_2',
                   'CLASSE_SOCIAL_SUBJECTIVA_1_7', 'SIT_LAB', 'FILLS', 'PERSONES_LLAR']

# Inglehart’s
columns_to_keep.extend(['POSTMAT_1A', 'POSTMAT_1B','POSTMAT_2A','POSTMAT_2B'])

# Schwartz’s
columns_to_keep.extend(['VHB_A_AUTODIRECCIO1', 'VHB_B_PODER1','VHB_C_UNIVERSALISME1',
                        'VHB_D_ASSOLIMENT1', 'VHB_E_SEGURETAT1', 'VHB_F_ESTIMULANT1',
                        'VHB_G_CONFORMITAT1', 'VHB_H_UNIVERSALISME2',
                        'VHB_I_TRADICIO1', 'VHB_J_HEDONISME1','VHB_K_AUTODIRECCIO2',
                        'VHB_L_BENEVOLENCIA1', 'VHB_M_ASSOLIMENT2', 'VHB_N_SEGURETAT2',
                        'VHB_O_ESTIMULANT2', 'VHB_P_CONFORMITAT3','VHB_Q_PODER2',
                        'VHB_R_BENEVOLENCIA2', 'VHB_S_UNIVERSALISME3', 'VHB_T_TRADICIO2', 'VHB_U_HEDONISME2'])

# Altres columnes que m'han semblat interessants
columns_to_keep.extend(['LLOC_NAIX', 'LLENGUA_IDENTIFICACIO','IMP_FAMILIA','IMP_AMICS',
                        'IMP_LLEURE', 'IMP_POLITICA','IMP_TREBALL','IMP_RELIGIO','SATIS_VIDA',
                        'CONFI_SOCIAL_WVS_1', 'SALUT','FELICITAT','ACTITUD_ECONOMIA','ACTITUD_IMMIGRACIO',
                        'ACTITUD_MEDIAMBIENT', 'DESIGUALTATS_GENERE','ECON_MERITOCRACIA','MEDIAMBIENT_CREIXEMENT',
                        'IMMIGRACIO_CULTURA', 'CONFI_POLICIA','CONFI_POLITICS','IDEOL_0_10','PERSONA_RELIGIOSA',
                        'DEPENDENCIA_ECO_PARES', 'INGRESSOS_1_15','INFO_POL_TV_FREQ','FREQ_CONSUM_INTERNET'])

valors_socials_cat = valors_socials_cat.loc[:, columns_to_keep]

# rename columns
new_column_names = {'DISTRICTE': 'D', 'SEXE': 'G', 'EDAT': 'DA', 'CIUTADANIA': 'N', 'ESTUDIS_1_6': 'E',
                    'ESTAT_CIVIL_2': 'S', 'CLASSE_SOCIAL_SUBJECTIVA_1_7': 'C', 'SIT_LAB': 'U','FILLS': 'Ch',
                    'PERSONES_LLAR': 'HR'}
valors_socials_cat.rename(columns=new_column_names, inplace=True)

# Neteja dades
values_to_remove = ['Valor perdut per error tècnic', 'Valor perdut per omissió de resposta', "Valor perdut per mode d'administració de l'enquesta"]
valors_socials_cat.replace(values_to_remove, np.nan, inplace=True)
valors_socials_cat.dropna(inplace=True)
print('Rows after cleaning: ', valors_socials_cat.shape[0])

def discretize_age(x):
    x = int(x)
    if x < 25:
        return 1
    elif x < 35:
        return 2
    elif x < 45:
        return 3
    elif x < 55:
        return 4
    elif x < 65:
        return 5
    else:
        return 6
valors_socials_cat['DA'] = valors_socials_cat['DA'].apply(discretize_age)

def discretize_llar(x):
    x = int(x)
    if x < 2:
        return 1
    elif x == 2:
        return 2
    else:
        return 3

valors_socials_cat['HR'] = valors_socials_cat['HR'].apply(discretize_llar)

def discretize_fills(x):
    x = int(x)
    if x == 0:
        return 1
    else:
        return 2

valors_socials_cat['Ch'] = valors_socials_cat['Ch'].apply(discretize_fills)

def discretize_inglehart_1(x):
    if x == 'Combatre la pujada dels preus' or x == 'Mantenir l’ordre en el país':
        return 1 # materialistes
    elif x == 'Protegir la llibertat d’expressió' or x == 'Augmentar la participació dels ciutadans en les decisions importants del govern':
        return 2 # post materialistes
    else:
        return None
    
def discretize_inglehart_2(x):
    if x == 'Una economia estable' or x == 'La lluita contra la delinqüència':
        return 1 # materialistes
    elif x == 'Avançar cap a una societat menys impersonal i més humana' or x == 'Avançar cap a una societat on les idees siguin més important que els diners':
        return 2 # post materialistes
    else:
        return None

valors_socials_cat['POSTMAT_1A'] = valors_socials_cat['POSTMAT_1A'].apply(discretize_inglehart_1)
valors_socials_cat['POSTMAT_1B'] = valors_socials_cat['POSTMAT_1B'].apply(discretize_inglehart_1)
valors_socials_cat['POSTMAT_2A'] = valors_socials_cat['POSTMAT_2A'].apply(discretize_inglehart_2)
valors_socials_cat['POSTMAT_2B'] = valors_socials_cat['POSTMAT_2B'].apply(discretize_inglehart_2)

# Encode values into a single variable
valors_socials_cat['Inglehart_index'] = (
    (valors_socials_cat['POSTMAT_1A'] - 1)*2 + # give more importance to the first selection
    (valors_socials_cat['POSTMAT_1B'] - 1) +
    (valors_socials_cat['POSTMAT_2A'] - 1)*2 +
    (valors_socials_cat['POSTMAT_2B'] - 1)
) # 0 -> full materialist, 6 -> full post-materialist

valors_socials_cat.drop(columns=['POSTMAT_1A', 'POSTMAT_1B', 'POSTMAT_2A', 'POSTMAT_2B'], inplace=True)

def discretize_ideo(x):
    x = int(x[0:2])
    if x < 4:
        return 1 # esquerra
    elif x < 7:
        return 2 # centre
    else:
        return 3 #dretes

valors_socials_cat['IDEOL_0_10'] = valors_socials_cat['IDEOL_0_10'].apply(discretize_ideo)

def discretize_des_gen(x):
    x = int(x[0:2])
    if x < 4:
        return 1 # perjudici homes
    elif x < 7:
        return 2 # centre
    else:
        return 3 # perjudici dones

valors_socials_cat['DESIGUALTATS_GENERE'] = valors_socials_cat['DESIGUALTATS_GENERE'].apply(discretize_des_gen)

def discretize_medi(x):
    if x == 'D’acord' or x=='Molt d’acord':
        return 1 # prioritat economia
    elif x == 'Ni d’acord ni en desacord':
        return 2 # centre
    elif x == 'En desacord' or x=='Molt en desacord':
        return 3 # prioritat medi ambient

valors_socials_cat['ACTITUD_MEDIAMBIENT'] = valors_socials_cat['ACTITUD_MEDIAMBIENT'].apply(discretize_medi)

def discretize_immi(x):
    if x == 'D’acord' or x=='Molt d’acord':
        return 1 # immigració dolenta
    elif x == 'Ni d’acord ni en desacord':
        return 2 # centre
    elif x == 'En desacord' or x=='Molt en desacord':
        return 3

valors_socials_cat['ACTITUD_IMMIGRACIO'] = valors_socials_cat['ACTITUD_IMMIGRACIO'].apply(discretize_immi)

def discretize_poli(x):
    x = int(x[0:2])
    if x < 4:
        return 1 # no confia
    elif x < 7:
        return 2 # centre
    else:
        return 3 # confia

valors_socials_cat['CONFI_POLICIA'] = valors_socials_cat['CONFI_POLICIA'].apply(discretize_poli)


def discretize_VHB(x):
    if x == 'No s’assembla a mi' or x=='No s’assembla gens a mi':
        return 1
    elif x == 'S’assembla poc a mi' or x=='S’assembla una mica a mi':
        return 2
    elif x == 'S’assembla a mi' or x=='S’assembla molt a mi':
        return 3

valors_socials_cat['VHB_A_AUTODIRECCIO1'] = valors_socials_cat['VHB_A_AUTODIRECCIO1'].apply(discretize_VHB)
valors_socials_cat['VHB_B_PODER1'] = valors_socials_cat['VHB_B_PODER1'].apply(discretize_VHB)
valors_socials_cat['VHB_C_UNIVERSALISME1'] = valors_socials_cat['VHB_C_UNIVERSALISME1'].apply(discretize_VHB)
valors_socials_cat['VHB_D_ASSOLIMENT1'] = valors_socials_cat['VHB_D_ASSOLIMENT1'].apply(discretize_VHB)
valors_socials_cat['VHB_E_SEGURETAT1'] = valors_socials_cat['VHB_E_SEGURETAT1'].apply(discretize_VHB)
valors_socials_cat['VHB_F_ESTIMULANT1'] = valors_socials_cat['VHB_F_ESTIMULANT1'].apply(discretize_VHB)
valors_socials_cat['VHB_G_CONFORMITAT1'] = valors_socials_cat['VHB_G_CONFORMITAT1'].apply(discretize_VHB)
valors_socials_cat['VHB_H_UNIVERSALISME2'] = valors_socials_cat['VHB_H_UNIVERSALISME2'].apply(discretize_VHB)
valors_socials_cat['VHB_I_TRADICIO1'] = valors_socials_cat['VHB_I_TRADICIO1'].apply(discretize_VHB)
valors_socials_cat['VHB_J_HEDONISME1'] = valors_socials_cat['VHB_J_HEDONISME1'].apply(discretize_VHB)
valors_socials_cat['VHB_K_AUTODIRECCIO2'] = valors_socials_cat['VHB_K_AUTODIRECCIO2'].apply(discretize_VHB)
valors_socials_cat['VHB_L_BENEVOLENCIA1'] = valors_socials_cat['VHB_L_BENEVOLENCIA1'].apply(discretize_VHB)
valors_socials_cat['VHB_M_ASSOLIMENT2'] = valors_socials_cat['VHB_M_ASSOLIMENT2'].apply(discretize_VHB)
valors_socials_cat['VHB_N_SEGURETAT2'] = valors_socials_cat['VHB_N_SEGURETAT2'].apply(discretize_VHB)
valors_socials_cat['VHB_O_ESTIMULANT2'] = valors_socials_cat['VHB_O_ESTIMULANT2'].apply(discretize_VHB)
valors_socials_cat['VHB_P_CONFORMITAT3'] = valors_socials_cat['VHB_P_CONFORMITAT3'].apply(discretize_VHB)
valors_socials_cat['VHB_Q_PODER2'] = valors_socials_cat['VHB_Q_PODER2'].apply(discretize_VHB)
valors_socials_cat['VHB_R_BENEVOLENCIA2'] = valors_socials_cat['VHB_R_BENEVOLENCIA2'].apply(discretize_VHB)
valors_socials_cat['VHB_S_UNIVERSALISME3'] = valors_socials_cat['VHB_S_UNIVERSALISME3'].apply(discretize_VHB)
valors_socials_cat['VHB_T_TRADICIO2'] = valors_socials_cat['VHB_T_TRADICIO2'].apply(discretize_VHB)
valors_socials_cat['VHB_U_HEDONISME2'] = valors_socials_cat['VHB_U_HEDONISME2'].apply(discretize_VHB)

# Encode values into a single variable
valors_socials_cat['Universalism'] = ((valors_socials_cat['VHB_C_UNIVERSALISME1'] - 1) +
                                     (valors_socials_cat['VHB_H_UNIVERSALISME2'] - 1) +
                                     (valors_socials_cat['VHB_S_UNIVERSALISME3'] - 1))

valors_socials_cat['Stimulation'] = ((valors_socials_cat['VHB_F_ESTIMULANT1'] - 1) +
                                     (valors_socials_cat['VHB_O_ESTIMULANT2'] - 1))

valors_socials_cat['Hedonism'] = ((valors_socials_cat['VHB_J_HEDONISME1'] - 1) +
                                     (valors_socials_cat['VHB_U_HEDONISME2'] - 1))

valors_socials_cat['Self-Direction'] = ((valors_socials_cat['VHB_A_AUTODIRECCIO1'] - 1) +
                                     (valors_socials_cat['VHB_K_AUTODIRECCIO2'] - 1))

valors_socials_cat['Achievement'] = ((valors_socials_cat['VHB_D_ASSOLIMENT1'] - 1) +
                                     (valors_socials_cat['VHB_M_ASSOLIMENT2'] - 1))

valors_socials_cat['Power'] = ((valors_socials_cat['VHB_B_PODER1'] - 1) +
                                     (valors_socials_cat['VHB_Q_PODER2'] - 1))

valors_socials_cat['Security'] = ((valors_socials_cat['VHB_E_SEGURETAT1'] - 1) +
                                     (valors_socials_cat['VHB_N_SEGURETAT2'] - 1))

valors_socials_cat['Conformity'] = ((valors_socials_cat['VHB_F_ESTIMULANT1'] - 1) +
                                     (valors_socials_cat['VHB_O_ESTIMULANT2'] - 1))

valors_socials_cat['Tradition'] = ((valors_socials_cat['VHB_I_TRADICIO1'] - 1) +
                                     (valors_socials_cat['VHB_T_TRADICIO2'] - 1))

valors_socials_cat['Benevolence'] = ((valors_socials_cat['VHB_L_BENEVOLENCIA1'] - 1) +
                                     (valors_socials_cat['VHB_R_BENEVOLENCIA2'] - 1))


valors_socials_cat.drop(columns=['VHB_A_AUTODIRECCIO1', 'VHB_B_PODER1','VHB_C_UNIVERSALISME1',
                                'VHB_D_ASSOLIMENT1', 'VHB_E_SEGURETAT1', 'VHB_F_ESTIMULANT1',
                                'VHB_G_CONFORMITAT1', 'VHB_H_UNIVERSALISME2',
                                'VHB_I_TRADICIO1', 'VHB_J_HEDONISME1','VHB_K_AUTODIRECCIO2',
                                'VHB_L_BENEVOLENCIA1', 'VHB_M_ASSOLIMENT2', 'VHB_N_SEGURETAT2',
                                'VHB_O_ESTIMULANT2', 'VHB_P_CONFORMITAT3','VHB_Q_PODER2',
                                'VHB_R_BENEVOLENCIA2', 'VHB_S_UNIVERSALISME3', 'VHB_T_TRADICIO2', 'VHB_U_HEDONISME2'], inplace=True)


C:\Users\mique\AppData\Local\Temp\ipykernel_15944\3133058008.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Rows after cleaning:  3054


In [2]:
from pgmpy.estimators import BicScore, MmhcEstimator, HillClimbSearch, ExpectationMaximization
from pgmpy.models import BayesianNetwork

est = HillClimbSearch(valors_socials_cat)
# Estimate the structure
VS_cat_structure_model = est.estimate(scoring_method=BicScore(valors_socials_cat))

VS_cat_structure_model = BayesianNetwork(VS_cat_structure_model.edges())
# We need to manually add independent variables, as they do no appear in the DAG
VS_cat_structure_model.fit(valors_socials_cat, estimator=ExpectationMaximization)

c:\Users\mique\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# create a dict to visualize the dependendies
depend_dict = {}
for connection in VS_cat_structure_model.edges():
    
    if not connection[0] in depend_dict.keys():
        depend_dict[connection[0]] = [connection[1]]
    else:
        depend_dict[connection[0]].append(connection[1])

nodes_in_DAG = set()
for edge in VS_cat_structure_model.edges():
    nodes_in_DAG.add(edge[0])
    nodes_in_DAG.add(edge[1])

# nodes_in_DAG imposes an ordering of variables for the plot
for feature in nodes_in_DAG:
    for key in depend_dict.keys():
        if key == feature:
            print(str(key)+" -> "+str(depend_dict[key]))

Benevolence -> ['Universalism']
DESIGUALTATS_GENERE -> ['G']
PERSONA_RELIGIOSA -> ['Tradition']
CONFI_SOCIAL_WVS_1 -> ['SATIS_VIDA', 'ECON_MERITOCRACIA', 'C']
N -> ['LLOC_NAIX']
HR -> ['DA', 'S']
IMP_POLITICA -> ['INFO_POL_TV_FREQ', 'ACTITUD_ECONOMIA']
DEPENDENCIA_ECO_PARES -> ['INGRESSOS_1_15', 'LLOC_NAIX', 'G']
IDEOL_0_10 -> ['DESIGUALTATS_GENERE', 'IMP_RELIGIO']
IMP_AMICS -> ['Benevolence', 'IMP_FAMILIA']
IMP_LLEURE -> ['IMP_AMICS']
ACTITUD_ECONOMIA -> ['ACTITUD_IMMIGRACIO']
Stimulation -> ['Conformity']
Conformity -> ['Hedonism']
LLOC_NAIX -> ['LLENGUA_IDENTIFICACIO']
DA -> ['DEPENDENCIA_ECO_PARES', 'E', 'IMP_TREBALL', 'U']
Power -> ['Achievement']
ACTITUD_MEDIAMBIENT -> ['MEDIAMBIENT_CREIXEMENT']
CONFI_POLITICS -> ['CONFI_POLICIA', 'IMP_POLITICA']
ACTITUD_IMMIGRACIO -> ['IMMIGRACIO_CULTURA', 'Inglehart_index', 'ACTITUD_MEDIAMBIENT', 'Security', 'CONFI_SOCIAL_WVS_1', 'DESIGUALTATS_GENERE']
Self-Direction -> ['Stimulation']
Hedonism -> ['Ch', 'Power', 'IMP_LLEURE']
SATIS_VIDA -> ['F

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import ExpectationMaximization

VS_model = BayesianNetwork(VS_cat_structure_model.edges())
VS_model.fit(valors_socials_cat, estimator=ExpectationMaximization)

  0%|          | 0/100 [00:03<?, ?it/s]


In [ ]:
from pgmpy.readwrite import BIFWriter

writer = BIFWriter(VS_model)
writer.write_bif('models/learned_model_cat_values.pgmx')

In [ ]:
from pgmpy.inference import VariableElimination

'''
AQUEST ÉS L'ESTUDI DE LA PROBABILITAT QUE ET VAIG FER INICIALMENT,
PERÒ QUE FINALMENT NO VAM UTILITZAR, AIXÍ QUE HO PODRIES BORRAR

NO FUNCIONA PERQUE DESPRÉS VAIG CANVIAR EL NOM D'ALGUNES VARIABLES
'''

infer = VariableElimination(VS_model)

materialista = {"POSTMAT_1A": 1 , "POSTMAT_1B": 1, "POSTMAT_2A": 1, "POSTMAT_2B": 1}
post_materialista = {"POSTMAT_1A": 2 , "POSTMAT_1B": 2, "POSTMAT_2A": 2, "POSTMAT_2B": 2}


q = infer.query(variables=["ACTITUD_IMMIGRACIO"], evidence=materialista)
print('materialista:\n', q)
q = infer.query(variables=["ACTITUD_IMMIGRACIO"], evidence=post_materialista)
print('post_materialista:\n', q)
print('######################################################')
q = infer.query(variables=["ACTITUD_MEDIAMBIENT"], evidence=materialista)
print('materialista:\n', q)
q = infer.query(variables=["ACTITUD_MEDIAMBIENT"], evidence=post_materialista)
print('post_materialista: ', q)
print('######################################################')
q = infer.query(variables=["CONFI_SOCIAL_WVS_1"], evidence=materialista)
print('materialista:\n', q)
q = infer.query(variables=["CONFI_SOCIAL_WVS_1"], evidence=post_materialista)
print('post_materialista:\n', q)
print('######################################################')
q = infer.query(variables=["DESIGUALTATS_GENERE"], evidence=materialista)
print('materialista:\n', q)
q = infer.query(variables=["DESIGUALTATS_GENERE"], evidence=post_materialista)
print('post_materialista:\n', q)
print('######################################################')
q = infer.query(variables=["IDEOL_0_10"], evidence=materialista)
print('materialista: ', q)
q = infer.query(variables=["IDEOL_0_10"], evidence=post_materialista)
print('post_materialista:\n', q)
print('######################################################')
q = infer.query(variables=["LLENGUA_IDENTIFICACIO"], evidence=materialista)
print('materialista:\n', q)
q = infer.query(variables=["LLENGUA_IDENTIFICACIO"], evidence=post_materialista)
print('post_materialista:\n', q)
print('######################################################')
q = infer.query(variables=["IMP_RELIGIO"], evidence=materialista)
print('materialista:\n', q)
q = infer.query(variables=["IMP_RELIGIO"], evidence=post_materialista)
print('post_materialista:\n', q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=materialista)
print('materialista:\n', q)
q = infer.query(variables=["FELICITAT"], evidence=post_materialista)
print('post_materialista:\n', q)


'''
if x == 'No s’assembla a mi' or x=='No s’assembla gens a mi':
    return 1
elif x == 'S’assembla poc a mi' or x=='S’assembla una mica a mi':
    return 2
elif x == 'S’assembla a mi' or x=='S’assembla molt a mi':
    return 3
'''

hedonisme_evidence = {"VHB_J_HEDONISME1": 3, "VHB_U_HEDONISME2": 3}
autodireccio_evidence = {"VHB_A_AUTODIRECCIO1": 3, "VHB_K_AUTODIRECCIO2": 3}
universalisme_evidence = {"VHB_C_UNIVERSALISME1": 3, "VHB_H_UNIVERSALISME2": 3, "VHB_S_UNIVERSALISME3": 3}
poder_evidence = {"VHB_B_PODER1": 3, "VHB_Q_PODER2": 3}
assoliment_evidence = {"VHB_D_ASSOLIMENT1": 3, "VHB_M_ASSOLIMENT2": 3}
seguretat_evidence = {"VHB_E_SEGURETAT1": 3, "VHB_N_SEGURETAT2": 3}
tradicio_evidence = {"VHB_I_TRADICIO1": 3, "VHB_T_TRADICIO2": 3}
estimulant_evidence = {"VHB_F_ESTIMULANT1": 3, "VHB_O_ESTIMULANT2": 3}
conformitat_evidence = {"VHB_G_CONFORMITAT1": 3, "VHB_P_CONFORMITAT3": 3}
benevolencia_evidence= {"VHB_L_BENEVOLENCIA1": 3, "VHB_R_BENEVOLENCIA2": 3}

hedonisme = ["VHB_J_HEDONISME1", "VHB_U_HEDONISME2"]
autodireccio = ["VHB_A_AUTODIRECCIO1", "VHB_K_AUTODIRECCIO2"]
universalisme = ["VHB_C_UNIVERSALISME1", "VHB_H_UNIVERSALISME2", "VHB_S_UNIVERSALISME3"]
poder = ["VHB_B_PODER1", "VHB_Q_PODER2"]
assoliment = ["VHB_D_ASSOLIMENT1", "VHB_M_ASSOLIMENT2"]
seguretat = ["VHB_E_SEGURETAT1", "VHB_N_SEGURETAT2"]
tradicio = ["VHB_I_TRADICIO1", "VHB_T_TRADICIO2"]
estimulant = ["VHB_F_ESTIMULANT1", "VHB_O_ESTIMULANT2"]
conformitat = ["VHB_G_CONFORMITAT1", "VHB_P_CONFORMITAT3"]
benevolencia= ["VHB_L_BENEVOLENCIA1", "VHB_R_BENEVOLENCIA2"]

q = infer.query(variables=tradicio, evidence={'IMP_RELIGIO': 'Molt important'})
print(q)
print('######################################################')
q = infer.query(variables=universalisme, evidence={'MEDIAMBIENT_CREIXEMENT':
    "S'ha de prioritzar el medi ambient malgrat que alenteixi el creixement econòmic, però han de convervar-se i/o augmentar"})
print(q)
print('######################################################')
q = infer.query(variables=benevolencia, evidence={'IMP_AMICS': 'Molt important'})
print(q)
print('######################################################')
q = infer.query(variables=seguretat, evidence={'ACTITUD_IMMIGRACIO': 1}) 
print(q)
print('######################################################')
q = infer.query(variables=["POSTMAT_2A"], evidence=universalisme_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["IMP_LLEURE"], evidence=hedonisme_evidence)
print(q)
print('######################################################')
q = infer.query(variables=benevolencia, evidence={'G': 'Femení'})
print(q)
print('######################################################')
q = infer.query(variables=benevolencia, evidence={'G': 'Masculí'})
print(q)
print('######################################################')
q = infer.query(variables=["Ch"], evidence=hedonisme_evidence)
print(q)
print('######################################################')
q = infer.query(variables=poder, evidence={'Ch': 2})
print(q)
print('######################################################')
q = infer.query(variables=["CONFI_POLICIA"], evidence=conformitat_evidence)
print(q)
print('######################################################')
q = infer.query(variables=universalisme, evidence={'ACTITUD_IMMIGRACIO': 1})
print(q)


q = infer.query(variables=["FELICITAT"], evidence=hedonisme_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=autodireccio_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=universalisme_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=poder_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=assoliment_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=seguretat_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=tradicio_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=estimulant_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=conformitat_evidence)
print(q)
print('######################################################')
q = infer.query(variables=["FELICITAT"], evidence=benevolencia_evidence)
print(q)

ValueError: Node POSTMAT_1A not in not in graph